# Lazy versus greedy evaluation

In [ ]:
%matplotlib inline

In [ ]:
%run notebook_setup.py

In [ ]:
import starry

starry.config.quiet = True

## tl;dr

Version `1.0` of the code evaluates things *lazily* by default, meaning that all internal values are nodes in a graph, stored as `theano` tensors. Lazy mode is required for interfacing with `pymc3` to do inference (refer to the several tutorials on `pymc3` sampling). If you *really* need the value of a `theano` object, you can always call its `eval()` method, but keep in mind that operation can be somewhat slow.

If, on the other hand, you're not interested in using `pymc3` or in any of the derivatives of `starry` models, you can disable lazy evaluation by typing

```python
starry.config.lazy = False
```

at the top of your script, *before* you instantiate any `starry` maps. If you do that, `starry` will behave as it did in previous versions: you don't have to call the `eval()` method or worry about any tensor nonsense.

## Lazy mode

One of the big changes in version `1.0` of `starry` is *lazy evaluation* mode, which is now the default. [Lazy evaluation](https://en.wikipedia.org/wiki/Lazy_evaluation) means that the evaluation of all expressions in the code is delayed until a numerical value is needed (i.e., when outputting or plotting the result). This is as opposed to [greedy or eager evaluation](https://en.wikipedia.org/wiki/Eager_evaluation), in which all expressions are evaluated on-the-fly, as soon as the code encounters them. In lazy evaluation mode, expressions are compiled and stored in memory as *nodes in a graph*, which are only executed when a numerical value is required. This strategy allows for some cool compile-time optimization under the hood. But by far the greatest advantage of lazy evaluation (at least in our case) is that it makes it easy to autodifferentiate expressions using backpropagation. This lets us compute derivatives of all expressions extremely efficiently, and those can be seemlessly integrated into derivative-based MCMC sampling schemes such as Hamiltonian Monte Carlo or NUTS.

Version `1.0` of `starry` is built on top of the [theano](https://github.com/Theano/Theano) machine learning library, which handles all of the graph compiling and backpropagation. There's lots of other software that does similar things (such as `tensorflow` and `pytorch`), but the advantage of `theano` is that it is also the backbone of [exoplanet](https://github.com/dfm/exoplanet) and [pymc3](https://github.com/pymc-devs/pymc3). This allows us to easily integrate `starry` with all the cool inference machinery of those two packages.

Let's look at some examples of how lazy evaluation works in `starry`. Let's instantiate a regular `starry` map:

In [ ]:
import starry

map = starry.Map(ydeg=1)

We can give this map a simple dipole by assigning a value to the coefficient of the $Y_{1,0}$ spherical harmonic:

In [ ]:
map[1, 0] = 0.5

Since the coefficient of the $Y_{0,0}$ harmonic is fixed at unity, our spherical harmonic coefficients are now the vector $y = (1, 0, \frac{1}{2}, 0)$. Here's what that looks like:

In [ ]:
map.show()

Recall that the spherical harmonic coefficients are stored in the `y` attribute of the map. Let's take a look:

In [ ]:
map.y

That doesn't look right, but it *is*: the vector $y$ is stored internally as a `theano` tensor and doesn't yet have a numerical value:

In [ ]:
type(map.y)

In order to access its value, I can call its `eval` method:

In [ ]:
map.y.eval()

Which is what we expected.

## Greedy mode

To run `starry` in greedy (i.e., not lazy) mode, you can add the following line somewhere near the top of your script:

In [ ]:
# *-*-*- DON'T DO THIS AT HOME! -*-*-*
# You shouldn't mix greedy and lazy maps in
# the same session, as you risk angering theano.
# I'm able to get away with it in this example
# because I'm just evaluating a few variables.
# But if I were to try to do anything else, things
# would probably break!
starry.config._allow_changes = True

In [ ]:
starry.config.lazy = False

(Note that if you try to change the evaluation mode after you've instantiated a `starry` map, the code will complain.)

In greedy mode, things behave as they did in previous versions of the code. Check it out:

In [ ]:
map = starry.Map(ydeg=1)

In [ ]:
map[1, 0] = 0.5

In [ ]:
map.y

In [ ]:
type(map.y)